In [3]:
# =============================================================================
#
# Pronostico time trend + fourier seasonal
#
# =============================================================================

#
# Carga de datos
#
import nbimporter  #  type: ignore
import functions  #  type: ignore
from pathlib import Path
import sys

homework_path = Path("C:/Users/crisc/Desktop/gitHub/2024-2-PRE-27-prediccion-basica-series-de-tiempo-Cristian-MarinUn/homework")
sys.path.append(str(homework_path))


df_orig = functions.load_data()
df_orig.head()

,yt_true
date,
1946-01-01,890
1946-02-01,992
1946-03-01,979
1946-04-01,959
1946-05-01,1110


In [4]:
#
# Carga de datos
#
import nbimporter  #  type: ignore
import functions  #  type: ignore


df_orig = functions.load_data()
df_orig.head()

,yt_true
date,
1946-01-01,890
1946-02-01,992
1946-03-01,979
1946-04-01,959
1946-05-01,1110


In [5]:
#
# Componentes de tendencia lineal
#
df_orig = functions.add_linear_trend_component(df_orig)
df_orig = functions.add_month_component(df_orig)
df_orig.head()

,yt_true,trend,month
date,,,
1946-01-01,890,0,1
1946-02-01,992,1,2
1946-03-01,979,2,3
1946-04-01,959,3,4
1946-05-01,1110,4,5


In [6]:

#
# Fourier
#
df_orig = functions.add_sin_cos_components(df_orig)
df_orig.head(15)

AttributeError: module 'functions' has no attribute 'add_sin_cos_components'

In [ ]:

# Particionamiento de los datos
#
(
    X_complete,
    y_complete,
    X_train,
    y_train,
    X_test,
    y_test,
) = functions.train_test_split(
    df=df_orig,
    x_columns=[
        "trend",
        "sin_12m",
        "cos_12m",
        "sin_6m",
        "cos_6m",
        "sin_4m",
        "cos_4m",
        "sin_3m",
        "cos_3m",
    ],
    y_column="yt_true",
)

In [ ]:

# Especificación del modelo
#
from sklearn.pipeline import Pipeline  #  type: ignore
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler  #  type: ignore
from sklearn.linear_model import LinearRegression  #  type: ignore
from sklearn.compose import ColumnTransformer  #  type: ignore

pipeline = Pipeline(
    [
        (
            "preprocessor",
            ColumnTransformer(
                [
                    (
                        "trend",
                        Pipeline(
                            [
                                ("poly", PolynomialFeatures(degree=3)),
                                ("scaler", MinMaxScaler()),
                            ]
                        ),
                        ["trend"],
                    ),
                ],
                remainder="passthrough",
            ),
        ),
        ("regressor", LinearRegression()),
    ]
)


In [ ]:

#
# Entrenamiento y pronostico
#
pipeline.fit(X_train, y_train)
df_orig["yt_pred_time_trend_3_fourier"] = pipeline.predict(X_complete)
functions.plot_time_series(df_orig)


In [ ]:

#
# Almacenamiento de los resultados
#
functions.save_forecasts(df_orig)


In [ ]:

#
# Métricas de error
#
metrics = functions.compute_evaluation_metrics(df_orig)
functions.save_metrics(metrics)
metrics